In [1]:
import sqlite3
import pandas as pd
import json
from progressbar import ProgressBar
from fastai.text import *

# Read Lablels

In [22]:
bs = 32

In [2]:
path = '../data/NELA2019/labels.csv'
labels_data = pd.read_csv(path)

In [3]:
labels_data.head()

,source,aggregated_label,"Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake",...,"Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stcenturywire,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,abcnews,0.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,...,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,activistpost,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,addictinginfo,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,adobochronicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
with open('../data/NELA2019/source-metadata.json') as file:
    metadata = (json.loads(file.read()))

In [5]:
for source in metadata.keys():
    labels_data.loc[labels_data.source == source, 'n_articles'] = metadata[source]['articles']

In [6]:
cols = ['source', 'n_articles', 'aggregated_label', 'Media Bias / Fact Check, label']
labels_data[cols].head(10)

,source,n_articles,aggregated_label,"Media Bias / Fact Check, label"
0,21stcenturywire,791.0,2.0,conspiracy_pseudoscience
1,abcnews,2861.0,0.0,left_center_bias
2,activistpost,2780.0,2.0,conspiracy_pseudoscience
3,addictinginfo,12.0,1.0,left_bias
4,adobochronicles,417.0,NaN,satire
5,ageofautism,646.0,2.0,conspiracy_pseudoscience
6,airwars,16.0,0.0,least_biased
7,aljazeera,11611.0,0.0,left_center_bias
8,americablognews,104.0,0.0,left_bias
9,anonnews,508.0,NaN,NaN


# Read Data

In [7]:
# Execute query and load results into pandas dataframe
def execute_query_pandas(path, query):
    conn = sqlite3.connect(path)
    df = pd.read_sql_query(query, conn)
    return df

In [8]:
path = '../data/NELA2019/nela-eng-2019.db'
query= ''' SELECT * FROM newsdata WHERE strftime('%m', date) in ('01')'''
#query= ''' SELECT * FROM newsdata '''

data = execute_query_pandas(path, query)

In [9]:
for source in labels_data['source']:
    data.loc[data.source == source, 'label'] = labels_data.loc[labels_data.source == source, 'aggregated_label'].values[0]
    data.loc[data.source == source, 'bias'] = labels_data.loc[labels_data.source == source, 'Media Bias / Fact Check, label'].values[0]

data = data.dropna(subset = ['label', 'bias'], how='any')

In [10]:
data.sample(3)

,id,date,source,title,content,author,url,published,published_utc,collection_utc,label,bias
17874,thetelegraph--2019-01-23--Property tycoon Nick...,2019-01-23,thetelegraph,Property tycoon Nick Candy buys fallen augment...,Property tycoon Nick Candy has acquired Blippa...,Matthew Field,https://www.telegraph.co.uk/technology/2019/01...,2019-01-23 12:41:09+00:00,1548265269,1567551193,1.0,right_center_bias
113973,thedailymirror--2019-01-16--How Ewan McGregors...,2019-01-16,thedailymirror,How Ewan McGregor's affair rocked family as da...,Ewan McGregor 's family found out about his af...,Emmeline Saunders,https://www.mirror.co.uk/3am/celebrity-news/br...,2019-01-16 13:56:46+00:00,1547665006,1567552238,1.0,left_bias
137903,theguardianuk--2019-01-12--Small energy firms ...,2019-01-12,theguardianuk,Small energy firms' customers warned over cred...,Consumers signed up to small energy firms are ...,Miles Brignall,https://www.theguardian.com/money/2019/jan/12/...,2019-01-12 07:45:33+00:00,1547297133,1567552814,0.0,left_center_bias


# Process Data

In [12]:
data_clean = (TextList.from_df(data, cols='content')
                                .split_by_rand_pct(0.1)
                                .label_from_df(cols='label')
                                .label_for_lm() 
                                .databunch(bs=32))

In [14]:
data_clean.save('data_lm_01.pkl')

In [15]:
data_clean.show_batch()

idx,text
0,11 ] xxbos [ 2 5 xxunk 5 ... 81 342 1969 11 ] xxbos [ 2 5 312 25 ... xxunk xxunk 11 26 ] xxbos [ 2 5 9 xxunk ... 104 0 xxunk 11 ] xxbos [ 2 6 225 92 ... 93 73 103 26 ] xxbos [ 2 5 9 xxunk ... 57 135 188 11 ] xxbos [ 2 16 144 74 ... 199
1,47 196 ... xxunk 136 xxunk 14 ] xxbos [ 2 5 9 5 ... 57 xxunk 11 26 ] xxbos [ 2 5 263 51 ... 88 xxunk 11 26 ] xxbos [ 2 5 44 71 ... 36 xxunk 11 26 ] xxbos [ 2 5 27 29 ... 64 9 xxunk 11 ] xxbos [ 2 16 5 245 ... 1815 151 xxunk 11 ] xxbos [
2,27 11 26 ] xxbos [ 2 5 9 5 xxunk xxunk 5 xxunk 5 xxunk 12 9 5 xxunk 10 46 49 737 ] xxbos [ 2 6 157 290 ... xxunk 5 xxunk 11 ] xxbos [ 2 16 5 xxunk ... xxunk xxunk xxunk 11 ] xxbos [ 2 5 9 5 ... 88 498 585 11 ] xxbos [ 2 5 9 5 ... 88 498
3,2 5 xxunk 5 ... xxunk xxunk 11 26 ] xxbos [ 2 5 xxunk 53 ... 341 5 xxunk 11 ] xxbos [ 2 16 82 139 ... xxunk 6 269 11 ] xxbos [ 2 5 xxunk 5 ... xxunk 5 xxunk 11 ] xxbos [ 2 5 176 67 ... 12 xxunk 11 30 ] xxbos [ 2 16 xxunk xxunk ... 5 322 539 11 ]
4,2 5 73 25 ... 109 97 xxunk 11 ] xxbos [ 2 5 17 9 ... 25 xxunk xxunk 11 ] xxbos [ 2 5 xxunk 16 ... 19 xxunk xxunk 11 ] xxbos [ 2 5 xxunk 11 ... 180 xxunk 11 26 ] xxbos [ 2 5 789 5 ... xxunk 209 359 11 ] xxbos [ 2 6 xxunk 6 ... 9 458 11 26 ]


In [16]:
data_clean.vocab.itos[30:40]

['"', 'he', 'as', 'i', 'said', 'at', 'be', 'has', 'have', 'are']

In [17]:
data_clean.train_ds[0][0]

Text xxbos [ 2 6 225 92 ... 16 xxunk xxunk 11 ]

In [21]:
data_clean.train_ds[0][0].data[:10]

array([    2,   170,   442,   682, 17072,  9227,   737,  1003,     0,     0])

In [25]:
data_lm = load_data('.', 'data_lm_01.pkl', bs=bs)

In [26]:
data_lm.show_batch()

idx,text
0,11 ] xxbos [ 2 5 xxunk 5 ... 81 342 1969 11 ] xxbos [ 2 5 312 25 ... xxunk xxunk 11 26 ] xxbos [ 2 5 9 xxunk ... 104 0 xxunk 11 ] xxbos [ 2 6 225 92 ... 93 73 103 26 ] xxbos [ 2 5 9 xxunk ... 57 135 188 11 ] xxbos [ 2 16 144 74 ... 199
1,47 196 ... xxunk 136 xxunk 14 ] xxbos [ 2 5 9 5 ... 57 xxunk 11 26 ] xxbos [ 2 5 263 51 ... 88 xxunk 11 26 ] xxbos [ 2 5 44 71 ... 36 xxunk 11 26 ] xxbos [ 2 5 27 29 ... 64 9 xxunk 11 ] xxbos [ 2 16 5 245 ... 1815 151 xxunk 11 ] xxbos [
2,27 11 26 ] xxbos [ 2 5 9 5 xxunk xxunk 5 xxunk 5 xxunk 12 9 5 xxunk 10 46 49 737 ] xxbos [ 2 6 157 290 ... xxunk 5 xxunk 11 ] xxbos [ 2 16 5 xxunk ... xxunk xxunk xxunk 11 ] xxbos [ 2 5 9 5 ... 88 498 585 11 ] xxbos [ 2 5 9 5 ... 88 498
3,2 5 xxunk 5 ... xxunk xxunk 11 26 ] xxbos [ 2 5 xxunk 53 ... 341 5 xxunk 11 ] xxbos [ 2 16 82 139 ... xxunk 6 269 11 ] xxbos [ 2 5 xxunk 5 ... xxunk 5 xxunk 11 ] xxbos [ 2 5 176 67 ... 12 xxunk 11 30 ] xxbos [ 2 16 xxunk xxunk ... 5 322 539 11 ]
4,2 5 73 25 ... 109 97 xxunk 11 ] xxbos [ 2 5 17 9 ... 25 xxunk xxunk 11 ] xxbos [ 2 5 xxunk 16 ... 19 xxunk xxunk 11 ] xxbos [ 2 5 xxunk 11 ... 180 xxunk 11 26 ] xxbos [ 2 5 789 5 ... xxunk 209 359 11 ] xxbos [ 2 6 xxunk 6 ... 9 458 11 26 ]


In [28]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [32]:
TEXT = "The new book"
N_WORDS = 100
N_SENTENCES = 2
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

The new book The Great American Bride of America ( 2014 ) , by James R. Brown , and The Gift of Marriage by Matthew Viability , the graphic novel by Martin Wallis , was a book that was based on The Lord of the Rings : The First World War and The War Within . In addition , the book of the same name was published in the United States by David
The new book The Strategy of Malcolm Sargent = = 
 
  The Malcolm of Autobiography is an anthology of poetry written by Malcolm Sargent in 1920 . The book explores the history of Malcolm x 's career and his early years in the US . The book was the first book of the Autobiography of a Generation Poet , which was first published in 1966 in Dublin . The book contains the first chapters in three chapters , including the earlier Autobiography .
